<table  align="left" width="100%"> <tr>
        <td  style="background-color:#ffffff;"><a href="https://qworld.net" target="_blank"><img src="../images/qworld.jpg" width="35%" align="left"></a></td>
        <td  align="right" style="background-color:#ffffff;vertical-align:bottom;horizontal-align:right">
            prepared by <a href="https://iitis.pl/pl/person/aglos" target="_blank"  >Adam Glos</a> and Özlem Salehi
        </td>        
</tr></table>

#  Grover Algorithm for Max-Cut Problem

Finally, we are ready to implement the Grover algorithm for the Max-Cut problem. 

For the initialization, we will apply Hadamard gate to the qubits representing the color of each vertex. We know how to implement the Grover diffusion operator. Now we need to implement an oracle which will "mark" the colorings in which there are $k$ edges whose endpoints are colored using different colors.

For the oracle, the procedure goes as follows:

* Implement edge checking for each edge (check whether an edge has endpoints with different colors).
* Sum the outputs of edge checking step.
* Check whether the sum is equal to (or greater) $k$ and save the flag on the auxillary qubit.
* Apply $Z$ gate on the auxillary qubit.

The oracle works for the decision version of the problem which checks whether there exists a coloring such that there are at least $k$ edges whose endpoints are colored using different colors. How to find the maximum size for the cut?

Suppose that $k$ is a three-digit number $b_2b_1b_0$. First, we should check whether there is a coloring such that $b_2$ is equal to 1. If it is, then we check if there is a coloring such that $b_2=1$ and $b_1=1$. Otherwise, we should check for the coloring such that $b_2=0$ and $b_1$=1. This way, by doing the binary search over binary representation, we can finally find the cut of maximal size.

Let us consider the following graph.
<img src="../images/path3.png" width="25%" align="center">

We will check whether there is a coloring such that two edges connect vertices with a different color. To check that we will verify, whether for number $b_1b_0$, $b_1$ is set to 1. We will use:
* Three qubits (0-2) for vertices.
* Two qubits (3-4) for edges.
* Two qubits (5-6) for storing the sum.
* Single qubit (7) for the auxillary qubit.

Since qubit 7 stores the flag, that is whether $b_1$ is set to 1 or not, we will apply the $Z$ gate on that qubit.

Below we present the final Grover algorithm. Note that there are two such solutions, so we apply the oracle only once.

In [1]:
import cirq
from cirq import H, Z, X, inverse, CX, CCX
s = cirq.Simulator()

def oracle_computation():
    #edge checking
    yield CX(qq[0], qq[3])
    yield CX(qq[1], qq[3])
    yield CX(qq[1], qq[4])
    yield CX(qq[2], qq[4])
    
    #adding qubit 3
    yield CX(qq[3], qq[5])
    
    #adding qubit 4
    yield CCX(qq[4], qq[5], qq[6])
    yield CX(qq[4], qq[5])
    
    #check if b1 is equal to 1 and store the result in the auxillary qubit
    yield CX(qq[6], qq[7])

def oracle():
    yield oracle_computation()
    yield Z(qq[7])
    yield inverse(oracle_computation())

def grover_diffusion():
    yield H.on_each(*(qq[:3]))
    yield X.on_each(*(qq[:3]))
    yield Z(qq[2]).controlled_by(qq[0], qq[1])
    yield X.on_each(*(qq[:3]))
    yield H.on_each(*(qq[:3]))


# the Grover algorithm
qq = cirq.LineQubit.range(8)
circuit = cirq.Circuit()
circuit.append(H.on_each(*(qq[0:3])))

for i in range(1):
    circuit.append(oracle())
    circuit.append(grover_diffusion())
    
circuit.append(cirq.measure(*qq[0:3], key='result'))

# determine the statistics of the measurements
trials_number = 10_000
samples = s.run(circuit, repetitions=trials_number)

print("Random guess probability:", 1./2**3)

# create an histogram of the result
def bitstring(bits):
    return "".join(str(int(b)) for b in bits)
counts = samples.histogram(key="result",fold_func=bitstring)
for state, c in counts.items():
    print("Probability of obsering", state, ": " ,c/trials_number)

Random guess probability: 0.125
Probability of obsering 101 :  0.5104
Probability of obsering 010 :  0.4896


We observe that the Grover's Search outputs 101 and 010, which are the two colorings where there are 2 edges connecting vertices with different colors. 

In the remaining of this notebook, you will apply the same algorithm for other graphs. 

### Task 1

Implement the Grover algorithm for the following graph
<img src="../images/completebipartite.png" width="25%" align="center">

and check whether there is a coloring in which there are exactly four edges connecting vertices with a different color. Apply the oracle twice.

In [1]:
import cirq
from cirq import H, Z, X, inverse, I, CX, CCX
s = cirq.Simulator()

# 0-3: vertices
# ?-?: edge checking
# ?-?: the number
# ?-?: auxillary

def edge_check(a, b, c):
    yield CX(qq[a], qq[c])
    yield CX(qq[b], qq[c])

def oracle_computation():
    # check all edges
    yield edge_check(0, 2, 4)
    yield edge_check(0, 3, 5)
    yield edge_check(1, 2, 6)
    yield edge_check(1, 3, 7)
    
    # add outputs of edge checking
    yield CX(qq[4],qq[8])
    
    yield CCX(qq[5], qq[8], qq[9])
    yield CX(qq[5],qq[8])
    
    yield CCX(qq[6], qq[8], qq[9])
    yield CX(qq[6],qq[8])
    
    yield X(qq[10]).controlled_by(qq[7], qq[8],qq[9])
    yield CCX(qq[7], qq[8], qq[9])
    yield CX(qq[7],qq[8])
    
    
    # check if number is equal to four 
    yield X(qq[8])
    yield X(qq[9])
    yield X(qq[11]).controlled_by(*(qq[8:11]))
    # note we don't have to undo the X gates!
    
def oracle():    
    yield oracle_computation()
    yield Z(qq[11])
    yield inverse(oracle_computation())
    
def grover_diffusion():
    yield H.on_each(*(qq[:4]))
    yield X.on_each(*(qq[:4]))
    yield Z(qq[3]).controlled_by(*(qq[:3]))
    yield X.on_each(*(qq[:4]))
    yield H.on_each(*(qq[:4]))


# the Grover algorithm
qq = cirq.LineQubit.range(12) 
circuit = cirq.Circuit()
circuit.append(H.on_each(*(qq[0:4])))

for i in range(2):
    circuit.append(oracle())
    circuit.append(grover_diffusion())
circuit.append(cirq.measure(*qq[0:4], key='result'))

# determine the statistics of the measurements
trials_number = 10_000
samples = s.run(circuit, repetitions=trials_number)
result = samples.measurements["result"]

print("Random guess probability:", 1/2**4)
# create an histogram of the result
def bitstring(bits):
    return "".join(str(int(b)) for b in bits)
counts = samples.histogram(key="result",fold_func=bitstring)
for state, c in counts.items():
    print("Probability of observing", state, ": " ,c/trials_number)

Random guess probability: 0.0625
Probability of observing 1100 :  0.4788
Probability of observing 0011 :  0.4653
Probability of observing 0010 :  0.0041
Probability of observing 0100 :  0.0045
Probability of observing 1111 :  0.0037
Probability of observing 1011 :  0.0047
Probability of observing 0000 :  0.0028
Probability of observing 1101 :  0.0044
Probability of observing 1000 :  0.0037
Probability of observing 0101 :  0.0046
Probability of observing 1110 :  0.0042
Probability of observing 0111 :  0.0041
Probability of observing 0110 :  0.0042
Probability of observing 0001 :  0.0037
Probability of observing 1001 :  0.0031
Probability of observing 1010 :  0.0041


### Task 2

Implement the Grover Algorithm for the following graph
<img src="../images/finalgrover1.png" width="25%" align="center">

and check whether there is coloring with six or more edges connecting vertices with a different color. Apply the oracle twice. 

*Hint*: Note that you may check only two bits of the number.

In [2]:
import cirq
from cirq import H, Z, X, inverse, CX, CCX
s = cirq.Simulator()

# 0-4: vertices
# 5-11: edge checking
# 12-14: the number
# 15: auxillary

def edge_check(a, b, c):
    yield CX(qq[a], qq[c])
    yield CX(qq[b], qq[c])

def oracle_computation():
    # check all edges
    yield edge_check(0, 1, 5)
    yield edge_check(0, 3, 6)
    yield edge_check(0, 4, 7)
    yield edge_check(1, 3, 8)
    yield edge_check(1, 4, 9)
    yield edge_check(2, 3, 10)
    yield edge_check(2, 4, 11)
    
    # add qubit 5   
    yield CX(qq[5], qq[12])
    
    # add qubits 6-7
    for j in range(6,8):
        yield CCX(qq[j], qq[12], qq[13])
        yield CX(qq[j], qq[12])
    
    # add qubits 8-11
    for j in range(8,12):
        yield X(qq[14]).controlled_by(qq[j], qq[12], qq[13])
        yield CCX(qq[j], qq[12], qq[13])
        yield CX(qq[j], qq[12])

    # set the last bit 
    # we do not have to check 12th qubit
    yield CCX(qq[13], qq[14], qq[15])
    
def oracle():    
    yield oracle_computation()
    yield Z(qq[15])
    yield inverse(oracle_computation())

def grover_diffusion():
    yield H.on_each(*(qq[:5]))
    yield X.on_each(*(qq[:5]))
    yield Z(qq[4]).controlled_by(*(qq[:4]))
    yield X.on_each(*(qq[:5]))
    yield H.on_each(*(qq[:5]))

# the Grover algorithm
qq = cirq.LineQubit.range(16) 
circuit = cirq.Circuit()
circuit.append(H.on_each(*(qq[0:5])))

for i in range(2):
    circuit.append(oracle())
    circuit.append(grover_diffusion())
    
circuit.append(cirq.measure(*qq[0:5], key='result'))

# determine the statistics of the measurements
trials_number = 10_000
samples = s.run(circuit, repetitions=trials_number)
result = samples.measurements["result"]

print("Random guess probability:", 1/2**5)
# create an histogram of the result
def bitstring(bits):
    return "".join(str(int(b)) for b in bits)
counts = samples.histogram(key="result",fold_func=bitstring)
for state, c in counts.items():
    print("Probability of observing", state, ": " ,c/trials_number)

Random guess probability: 0.03125
Probability of observing 00011 :  0.454
Probability of observing 11100 :  0.4573
Probability of observing 00100 :  0.0036
Probability of observing 10101 :  0.0038
Probability of observing 10010 :  0.0029
Probability of observing 00111 :  0.0025
Probability of observing 01110 :  0.0032
Probability of observing 01111 :  0.0019
Probability of observing 00000 :  0.0029
Probability of observing 00110 :  0.0034
Probability of observing 00101 :  0.0027
Probability of observing 01101 :  0.0027
Probability of observing 01011 :  0.0023
Probability of observing 11001 :  0.0033
Probability of observing 11011 :  0.0026
Probability of observing 11000 :  0.0039
Probability of observing 00001 :  0.0032
Probability of observing 10000 :  0.0038
Probability of observing 01010 :  0.003
Probability of observing 00010 :  0.0029
Probability of observing 01100 :  0.0029
Probability of observing 11101 :  0.0036
Probability of observing 01001 :  0.0029
Probability of observing 

## Final remarks

We have successfully implemented the Grover's Search algorithm for the Max-Cut problem. Congratulations! However, there are several interesting additional facts.

Currently for each edge, we have a separate qubit. We could reuse a single qubit at the cost of additional quantum gates. While the circuit would be longer, we could save some qubits.

Next, for each task, the number of iterations was given. While for bipartiteness checking we know there are always exactly two answers, this may not be the case for the Max-Cut problem. Fortunately, there is a *quantum counting algorithm*, which can efficiently determine the number of solutions and hence the number of iterations.